###  Pruebas de API para el Modelo de Predicción de Accidentes en Minería

Este notebook forma parte del trabajo final del curso de MLOps  y está diseñado para probar el modelo de Machine Learning a través de su API en **Databricks Model Serving**.

El objetivo es verificar que el modelo responde correctamente a solicitudes de predicción mediante llamadas HTTP a la API.  
Las pruebas incluyen:

- **Carga de la API y autenticación** con un token de acceso.
- **Prueba de inferencia con una sola fila de datos** para validar respuestas individuales.
- **Prueba con múltiples filas** para evaluar el rendimiento con varias predicciones simultáneas.
- **Medición del tiempo de respuesta** para analizar la eficiencia del servicio.

Estas pruebas son fundamentales dentro de **MLOps**, ya que aseguran que el modelo puede integrarse correctamente con aplicaciones externas y responder eficientemente a solicitudes en tiempo real.

<style>
    p { text-align: justify; }
</style>


### 1. Importación de librerías y configuración del entorno
Se importan las librerías necesarias para realizar pruebas de inferencia a través de la API del modelo desplegado en Databricks Model Serving.
Esto permitirá evaluar si el modelo responde correctamente en un entorno de producción.


In [0]:
# Paso 1: Importar librerías
import requests
import json
import time
import pandas as pd

print("Notebook 05 (API_Test) - Iniciando...")


Notebook 05 (API_Test) - Iniciando...


### 2. Configuración del endpoint de la API y autenticación
Se define la URL de la API donde se encuentra el modelo en Databricks y se configura el token de autenticación. Este token es necesario para autorizar las solicitudes y acceder al servicio de inferencia.

In [0]:
# Paso 2: Configurar endpoint y token

API_URL = "https://adb-1781258311325241.1.azuredatabricks.net/serving-endpoints/accidentprediction2025-endpoint/invocations"
DATABRICKS_TOKEN = "********************" 

headers = {
    "Authorization": f"Bearer {DATABRICKS_TOKEN}",
    "Content-Type": "application/json"
}

print("API_URL =", API_URL)


API_URL = https://adb-1781258311325241.1.azuredatabricks.net/serving-endpoints/accidentprediction2025-endpoint/invocations


### 3. Prueba de inferencia con una sola fila de datos
Se envía una sola instancia de prueba a la API para verificar que el modelo responde correctamente. Se mide el tiempo de respuesta y se valida si la predicción se obtiene sin errores.


In [0]:
# Paso 3: Enviar datos de prueba a la API

sample_data_single = {
    "dataframe_split": {
        "columns": [
            "Shift_Type", 
            "Weather_Risk_Index", 
            "Job_Risk_Level", 
            "Hours_Worked",
            "Employee_Experience_Years", 
            "Safety_Violations", 
            "Inspection_Frequency", 
            "Temperature_C"
        ],
        "data": [
            [1.0, 3.5, 2.0, 8.0, 5.0, 0.0, 3.0, 25.0]
        ]
    }
}

try:
    start_time = time.time()
    response = requests.post(API_URL, headers=headers, json=sample_data_single)
    response_time = time.time() - start_time

    if response.status_code == 200:
        print("✅ API Respondió correctamente (single).")
        print("Predicción (single):", response.json())
    else:
        print("❌ Error en la respuesta (single):", response.status_code, response.text)

    print(f"⏱ Tiempo de respuesta: {response_time:.2f} segundos")

except Exception as e:
    print("🚨 Error al conectar con la API (single):", str(e))


✅ API Respondió correctamente (single).
Predicción (single): {'predictions': [1.0]}
⏱ Tiempo de respuesta: 1.54 segundos


### 4. Prueba de inferencia con múltiples filas de datos
Se prueba la API con varias instancias al mismo tiempo para evaluar la capacidad del modelo de procesar múltiples solicitudes en paralelo.
Esto es útil para verificar el rendimiento del modelo en escenarios reales con grandes volúmenes de datos.

In [0]:
# Paso 4: Varias filas de test
sample_data_multi = {
    "dataframe_split": {
        "columns": [
            "Shift_Type", 
            "Weather_Risk_Index", 
            "Job_Risk_Level", 
            "Hours_Worked",
            "Employee_Experience_Years", 
            "Safety_Violations", 
            "Inspection_Frequency", 
            "Temperature_C"
        ],
        "data": [
            [2.0, 1.5, 1.0, 10.0, 2.0, 1.0, 2.0, 15.0],
            [3.0, 4.5, 3.0, 6.0,  8.0, 2.0, 4.0, 20.0],
            [1.0, 2.5, 2.0, 7.0,  3.0, 0.0, 4.0, 22.0]
        ]
    }
}

try:
    start_time = time.time()
    response = requests.post(API_URL, headers=headers, json=sample_data_multi)
    response_time = time.time() - start_time

    if response.status_code == 200:
        print("✅ API Respondió correctamente (multi).")
        preds_multi = response.json()
        print("Predicciones (multi):", preds_multi)
    else:
        print("❌ Error en la respuesta (multi):", response.status_code, response.text)

    print(f"⏱ Tiempo de respuesta: {response_time:.2f} segundos")

except Exception as e:
    print("🚨 Error al conectar con la API (multi):", str(e))


✅ API Respondió correctamente (multi).
Predicciones (multi): {'predictions': [1.0, 1.0, 1.0]}
⏱ Tiempo de respuesta: 0.95 segundos
